# ABS Monthly Labour Force 6202

## Python set-up

In [1]:
# analytic imports
import matplotlib.pyplot as plt
import pandas as pd

# local imports
from abs_data_capture import (
    # clear_cache,
    AbsLandingPage,
    get_abs_data,
    get_fs_constants,
    get_identifier,
    metacol,
    get_plot_constants,
    plot_rows_collectively,
    plot_rows_individually,
    plot_rows_seas_trend,
    find_rows,
    iudts_from_row,
)
from abs_summary_plot import plot_summary
from plotting import (
    calc_and_plot_growth,
    clear_chart_dir,
    line_plot,
    plot_covid_recovery,
    set_chart_dir,
    state_abbr,
    state_colors,
)

# pandas display settings
pd.options.display.max_rows = 9999
pd.options.display.max_columns = 999

# display all charts in this notebook ...
SHOW = False

## Get data from ABS

In [2]:
LANDING_PAGE = AbsLandingPage(
    theme="labour",
    parent_topic="employment-and-unemployment",
    topic="labour-force-australia",
)

In [3]:
def grab_data(
    landing_page: AbsLandingPage,
    zip_table: int = 0,
    verbose: bool = False,
    inspect: str = "",
    use_selenium: bool = False,
) -> tuple[dict, str, dict, pd.Timestamp, list[None | pd.Timestamp], tuple[str, str]]:
    """Get ABS data, initialise plot directory and set plot style."""

    # get the data
    abs_dict_ = get_abs_data(
        landing_page=landing_page,
        zip_table=zip_table,
        verbose=verbose,
        inspect=inspect,
    )
    source_, chart_dir, _cat_id, meta_ = get_fs_constants(abs_dict_, landing_page)
    recent_, plot_times_, plot_tags_ = get_plot_constants(meta_)

    # fix up chart directory
    set_chart_dir(chart_dir)
    clear_chart_dir(chart_dir)
    plt.style.use("fivethirtyeight")

    return abs_dict_, source_, meta_, recent_, plot_times_, plot_tags_


abs_dict, source, meta, recent, plot_times, plot_tags = grab_data(LANDING_PAGE)

## Plot

### Labour force summary chart

In [4]:
required  = {
    # order from least to most important (ie. bottom to top)
    # chart label: [ABS code, growth over n periods]
    "1 month growth: Civilian pop": ["A84423091W", 1],
    "12 month growth: Civilian pop": ["A84423091W", 12],
    "1 month growth: Num underemployed": ["A85255719L", 1],
    "12 months growth: Num underemployed": ["A85255719L", 12],
    "1 month growth: Num employed": ["A84423043C", 1],
    "12 months growth: Num employed": ["A84423043C", 12],
    "1 month growth: Num unemployed": ["A84423046K", 1],
    "12 months growth: Num unemployed": ["A84423046K", 12],
    "1 month growth: Num hours worked": ["A84426277X", 1],
    "12 month growth: Num hours worked": ["A84426277X", 12],
    "Underemployment ratio": ["A85255724F", 0],
    "Participation rate": ["A84423051C", 0],
    "Employment to pop ratio": ["A84423054K", 0],
    "Unemployment rate": ["A84423050A", 0],
}
    

start = "2000-01"
plot_summary(
    required, 
    abs_dict, 
    meta,
    start=start,
    title=f"Key labour force statistics {abs_dict['1'].index[-1]}",
    rfooter=source,
    lfooter="Australia. All seasonally adjusted except civilian population. "
            "All values are percentages",
    show=SHOW,
    verbose=False,
)

Overriding x0: True


### Headline seasonally adjusted/trend charts

In [5]:
def headline_charts():
    """Produce headline charts."""

    selector1 = {
        "1": metacol.table,
        "Persons": metacol.did,
    }

    selector2 = {
        "Monthly hours worked in all jobs ;  Persons ;": metacol.did,
        "19": metacol.table,
    }

    for selector in (selector1, selector2):
        plot_rows_seas_trend(
            abs_dict,
            selector,
            starts=plot_times,
            tags=plot_tags,
            rfooter=source,
            lfooter="Australia. ",
            verbose=False,
            show=SHOW,
        )


headline_charts()

### Headline COVID recovery charts

In [6]:
def covid_charts():
    """Produce pre/post COVID trajectory charts."""

    service_type = "Seasonally Adjusted"
    selector1 = {
        "Persons": metacol.did,
        "1": metacol.table,
        service_type: metacol.stype,
    }
    selector2 = {
        "Monthly hours worked in all jobs ;  Persons ;": metacol.did,
        "19": metacol.table,
        service_type: metacol.stype,
    }
    selector3 = {
        "Original": metacol.stype,
        "1": metacol.table,
        "Civilian population aged 15 years and over ;  Persons ;": metacol.did,
    }

    for selector in (selector1, selector2, selector3):
        plot_rows_individually(
            abs_dict,
            selector=selector,
            plot_function=plot_covid_recovery,
            tags="COVID",
            rfooter=source,
            lfooter="Australia. ",
            show=SHOW,
        )


covid_charts()

### State comparisons 

In [7]:
# use linestyle to help differentiate datasets ...
linestyle = {
    "style": [
        "-.",
        "-",
        "--",
        ":",
    ]
    * 3  # repeat enough to cover all series
}

In [8]:
def state_charts():
    """Produced trend charts for states."""

    states = (
        "New South Wales|Victoria|Queensland|South Australia"
        "|Western Australia|Tasmania|Northern Territory|Australian Capital Territory"
    )
    # service_type = "Seasonally Adjusted"
    service_type = "Trend"
    selector = {
        states: metacol.did,
        service_type: metacol.stype,
    }
    plot_stems = [
        "Unemployment rate ;  Persons",
        "Participation rate ;  Persons",
    ]

    for stem in plot_stems:
        xselector = {**selector, stem: metacol.did}
        plot_rows_collectively(
            abs_dict,
            selector=xselector,
            regex=True,
            starts=recent,
            width=2,
            **linestyle,
            legend={"loc": "best"},
            rfooter=source,
            lfooter=f"Australia. {service_type.capitalize()} series.",
            show=SHOW,
        )


state_charts()

In [9]:
def state_employment_growth():
    """Employed persions compared with the 2019 average / index = 100."""

    states = (
        "New South Wales|Victoria|Queensland|South Australia"
        "|Western Australia|Tasmania|Northern Territory|Australian Capital Territory"
    )
    selector = {
        states: metacol.tdesc,
    }
    plot_stems = [
        ["Employed total ;  Persons ;", "Trend"],
        ["Unemployed total ;  Persons ;", "Trend"],
        ["Civilian population aged 15 years and over ;", "Original"],
    ]

    for stem, service_type in plot_stems:
        xselector = {
            **selector,
            stem: metacol.did,
            service_type: metacol.stype,
        }
        frame = pd.DataFrame()
        for _, row in find_rows(
            meta,
            xselector,
            exact=False,
            regex=True,
            verbose=False,
        ).iterrows():
            series_id, units, did, table, _ = iudts_from_row(row)
            state = row[metacol.tdesc].split(" - ")[0]
            state = state.split(", ")[-1]
            frame[state] = abs_dict[table][series_id]
        if len(frame) == 0:
            continue

        for comparison_year in (2019, 2022):
            base = frame.loc[frame.index.year == comparison_year].mean()
            index_df = frame.divide(base) * 100
            index_df = index_df.loc[index_df.index.year >= comparison_year]
            index_df.columns = index_df.columns.map(state_abbr)

            title = f"{stem} cf {comparison_year} Ave.".replace(" ;", ";").replace(
                "  ", " "
            )
            line_plot(
                index_df,
                title=title,
                ylabel=f"Index ({comparison_year} = 100)",
                color=[state_colors[x] for x in index_df.columns],
                axhline={"y": 100, "lw": 0.75, "c": "#555555"},
                legend={"loc": "best", "fontsize": "x-small", "ncols": 2},
                **linestyle,
                rfooter=source,
                lfooter=f"Australia. {service_type.capitalize()} series.",
                width=2,
                show=SHOW,
            )


state_employment_growth()

### Headline Growth

In [10]:
def growth_charts(plot_stems: str, service_type: str):
    """Produce growth charts."""

    plot_rows_individually(
        abs_dict,
        # data selection
        selector={
            "1": metacol.table,
            plot_stems: metacol.did,
            service_type: metacol.stype,
        },
        regex=True,
        # plotting
        plot_function=calc_and_plot_growth,
        from_=plot_times,
        tag="growth",
        rfooter=source,
        lfooter=f"Australia. {service_type.capitalize()} series. ",
        show=SHOW,
    )


def chart_growth():
    """Structure/select growth charts for plotting."""

    # Seasonally adjusted series
    plot_stems = "|".join(
        [
            "Employed total ;  Persons ;",
            "Unemployed total ;  Persons ;",
            "Labour force total ;  Persons ;",
        ]
    )
    service_type = "Seasonally Adjusted"
    growth_charts(plot_stems, service_type)

    # Original series
    plot_stems = "|".join(
        [
            "Civilian population aged 15 years and over ;  Persons ;",
        ]
    )
    service_type = "Original"
    growth_charts(plot_stems, service_type)


chart_growth()

### Gendered comparisons

In [11]:
def gender_charts():
    """Produce gender comparison charts."""

    genders = ["Males", "Females"]
    colours = ["cornflowerblue", "hotpink"]

    token = "by gender"  # note: also appears in plot title
    plots = {
        # data description                       # table
        f"Employed total ;  > {token} ;": "1",
        f"> Employed part-time ;  > {token} ;": "1",
        f"> Employed full-time ;  > {token} ;": "1",
        f"Unemployed total ;  > {token} ;": "1",
        f"Labour force total ;  > {token} ;": "1",
        f"Unemployment rate ;  > {token} ;": "1",
        f"Participation rate ;  > {token} ;": "1",
        f"Monthly hours worked in all jobs ;  > {token} ;": "19",
    }

    series_type = "Seasonally Adjusted"

    for plot, table in plots.items():
        # assemble data
        collection = {}
        for gender in genders:
            this_gender = plot.replace(token, gender)
            ident, units = get_identifier(meta, this_gender, series_type, table)
            collection[gender] = abs_dict[table][ident]
        df = pd.DataFrame(collection)

        # Hours is a special case
        if units in ["000 Hours", "Thousand Hours"]:
            units = "Billion Hours"
            df = df / 1_000_000

        title = f"Australia: {plot}".replace(" ;", "").replace("  ", " ")
        line_plot(
            df,
            starts=plot_times,
            tags=plot_tags,
            color=colours,
            title=title,
            ylabel=units,
            rfooter=f"{source} {table}",
            lfooter=f"{series_type.capitalize()} series",
            show=SHOW,
        )


gender_charts()

## Finished

In [12]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Mon Jun 17 2024 21:17:57

Python implementation: CPython
Python version       : 3.12.3
IPython version      : 8.25.0

pandas    : 2.2.2
matplotlib: 3.8.4

Watermark: 2.4.3



In [13]:
print("Finished")

Finished
